# PV Degradation Tools Tutorial - 2
### Degradation and Solder Fatigue
***
Matthew Brown

2023.06.02
***

**Requirements**:
- compatible weather file (PSM3, TMY3, EPW)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
- Activation energies for test material [kJ/mol]

**Objectives**:
1. Read in the weather data
2. Generate basic modeling data
3. Calculate VantHoff degradation acceleration factor
4. Calculate Arrhenius degradation acceleration factor
5. Solder Fatigue

In [1]:
import os
import pandas as pd

import pvdeg 
from pvdeg import TEST_DATA_DIR

## 1. Read-In the Weather File

This is usually the first step. Use a PSM3, TMY3, or EPW file. For this demo, use the provided PSM3 weather file.

In [2]:
PSM_FILE = os.path.join(TEST_DATA_DIR,'psm3_pytest.csv')
WEATHER, META = pvdeg.weather.read(PSM_FILE,'psm')

## 2. Generate Basic Modeling Data

For this tutorial we will need solar position, POA, and PV cell temperature.

In [8]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, solar_position=sol_pos)

temp_cell = pvdeg.temperature.cell(weather_df=WEATHER, poa=poa_df)

## 3. VantHoff Degradation

In [ ]:
# chamber irradiance (W/m^2)
I_chamber = 1000 
# chamber temperature (C)
temp_chamber = 60

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.Degradation.vantHoff_deg(I_chamber=I_chamber, poa_global=poa_global,
                                            temp_cell=temp_cell, temp_chamber=temp_chamber)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.Degradation.IwaVantHoff(poa_global=poa_global, temp_cell=temp_cell)

## Arrhenius

In [ ]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

# calculate the module temperature with pvlib (see documentation for help)
temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=data['Temperature'],
                                            wind_speed=data['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

rh_surface = pvdeg.StressFactors.rh_surface_outside(rh_ambient=data['Relative Humidity'],
                                                        temp_ambient=data['Temperature'],
                                                        temp_module=temp_module)

arrhenius_deg = pvdeg.Degradation.arrhenius_deg(I_chamber=I_chamber, rh_chamber=rh_chamber,
                                              rh_outdoor=rh_surface, poa_global=poa_global,
                                              temp_chamber=temp_chamber, temp_cell=temp_cell,
                                              Ea=Ea)

irr_weighted_avg_a = pvdeg.Degradation.IwaArrhenius(poa_global=poa_global, 
                                                  rh_outdoor=data['Relative Humidity'],
                                                  temp_cell=temp_cell, Ea=Ea)